<center>

# Web Intelligence

# Who will win the Tennis Australian Open 2020?

## 03 Data Cleaning - Categorical Data Transformation

### Riccardo Spolaor (864877)

</center>

### Trasformazione Variabili Categoriali
Controllo solo le feature categoriali (Tipo = *Object*) e procedo alla loro trasformazione in variabili numeriche.

In [4]:
import pandas

In [1]:
%store -r
%store -z

In [2]:
print ("Variabili Categoriali:")
print(dataframe.dtypes[dataframe.dtypes == object].index.values)

for value in dataframe.dtypes[dataframe.dtypes == object].index.values:
    print('\n', value, 'values length: ', len(dataframe[value].unique()))
    print('\t', dataframe[value].unique()[:20])

Variabili Categoriali:
['Location' 'Tournament' 'Series' 'Court' 'Surface' 'Round' 'Winner'
 'Loser' 'Comment']

 Location values length:  111
	 ['Adelaide' 'Chennai' 'Doha' 'Auckland' 'Sydney' 'Melbourne' 'Milan'
 'Copenhagen' 'Marseille' 'Vina del Mar' 'Buenos Aires' 'Memphis'
 'Rotterdam' 'Acapulco' 'Dubai' 'San Jose' 'Delray Beach' 'Scottsdale'
 'Indian Wells' 'Miami']

 Tournament values length:  208
	 ['AAPT Championships' 'TATA Open' 'Qatar Open' 'Heineken Open'
 'adidas International' 'Australian Open' 'Milan Indoors'
 'Copenhagen Open' 'Marseille Open' 'Bellsouth Open' 'Copa AT&T'
 'Kroger St. Jude' 'ABN AMRO World Tennis Tournament' 'Abierto Mexicano'
 'Dubai Championships' 'Sybase Open' 'Citrix Tennis Championships'
 'Franklin Templeton Tennis Classic' 'Indian Wells TMS' 'Ericsson Open']

 Series values length:  8
	 ['International' 'Grand Slam' 'International Gold' 'Masters' 'Masters Cup'
 'ATP250' 'ATP500' 'Masters 1000']

 Court values length:  2
	 ['Outdoor' 'Indoor']

 

Uso la tecnica dello ***one-hot-encoding*** per trasformare le feature con un numero limitato di label e che non presentano nessun tipo di struttura gerarchica (***Court***, ***Surface***, ***Comment***).

In [5]:
dataframe = pandas.get_dummies(dataframe, prefix = ['', '', ''],
                   prefix_sep = '', columns = ['Court', 'Surface', 'Comment'])

dataframe.columns

Index(['csvID', 'ATP', 'Location', 'Tournament', 'Date', 'Series', 'Round',
       'Best of', 'Winner', 'Loser', 'WPlayed', 'LPlayed', 'WWinRatio',
       'LWinRatio', 'WLossRatio', 'LLossRatio', 'WRank', 'WRankAdded', 'LRank',
       'LRankAdded', 'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5',
       'L5', 'Wsets', 'Lsets', 'WPts', 'WPtsAdded', 'LPts', 'LPtsAdded',
       'MaxW', 'MaxL', 'AvgW', 'AvgL', 'MinW', 'MinL', 'Indoor', 'Outdoor',
       'Carpet', 'Clay', 'Grass', 'Hard', 'Awarded', 'Completed',
       'Disqualified', 'Retired', 'Sched', 'Walkover'],
      dtype='object')

Per quanto riguarda i ***Round***:
* Posso assegnare alle feature un valore numerico crescente in base all'importanza del Round:
    * *1st Round* < *2nd Round* < *3rd Round* < *4th Round* < *Quarterfinals* < *Semifinals* < *The Final*;
* *Round Robin* non è paragonabile agli altri tipi di round, quindi aggiungo una colonna con valore:
    * 0 se il Round non è *Round Robin*;
    * 1 se lo è.

In [6]:
print('Tipi di Round:')

print(dataframe['Round'].unique())

dataframe.insert(dataframe.columns.get_loc('Round') + 1, column = 'RoundRobin', value = 0)
dataframe.loc[dataframe['Round'] == 'Round Robin', 'RoundRobin'] = 1

roundsOrdered = {'1st Round': 1, '2nd Round': 2, 'Quarterfinals': 5, 'The Final': 7, 'Semifinals': 6,
 '3rd Round': 3, '4th Round': 4, 'Round Robin': 0}

dataframe['Round'] = dataframe['Round'].apply(lambda x : roundsOrdered[x])

Tipi di Round:
['1st Round' '2nd Round' 'Quarterfinals' 'The Final' 'Semifinals'
 '3rd Round' '4th Round' 'Round Robin']


Per quanto riguarda la colonna ***Series***:

* Posso assegnare alle feature un valore numerico crescente in base all'importanza del tipo di torneo;
* Alcuni tornei hanno cambiato nome nel tempo e risulta:
    * *ATP 250* = *International* < *ATP 500* = *International Gold* < *Masters* = *Masters 1000* < *Masters Cup* < *Grand Slam*

In [7]:
print('Tipi di Tornei:')

print(dataframe['Series'].unique())

pandas.DataFrame(columns = ['Series','WPts'], data = [
    ['Grand Slam', 2000],
    ['Masters Cup/ATP Finals', 1500],
    ['ATP250/International', 250],
    ['ATP500/International Gold', 500],
    ['Masters/Masters 1000', 1000],
])

Tipi di Tornei:
['International' 'Grand Slam' 'International Gold' 'Masters' 'Masters Cup'
 'ATP250' 'ATP500' 'Masters 1000']


,Series,WPts
0,Grand Slam,2000
1,Masters Cup/ATP Finals,1500
2,ATP250/International,250
3,ATP500/International Gold,500
4,Masters/Masters 1000,1000


In [8]:
seriesOrdered = {'ATP250': 1, 'International': 1, 'ATP500': 2,'International Gold': 2, 'Masters' : 3, 'Masters 1000': 3,
                'Masters Cup': 4, 'Grand Slam': 5}
dataframe['Series'] = dataframe['Series'].apply(lambda x : seriesOrdered[x])

Le feature ***Tournament*** e ***Location*** presentano troppi label distinti.
Per non allargare troppo il Dataframe con lo ***one-hot-encoding*** uso il ***Label Encoder*** per trasformare ogni stringa in un valore intero.

In [9]:
from sklearn.preprocessing import LabelEncoder

leTournament = LabelEncoder().fit(dataframe['Tournament'])
leLocation = LabelEncoder().fit(dataframe['Location'])

dataframe['Tournament'] = leTournament.transform(dataframe['Tournament'])
dataframe['Location'] = leLocation.transform(dataframe['Location'])

In [16]:
%store leTournament
%store leLocation

%store

Stored 'leTournament' (LabelEncoder)
Stored 'leLocation' (LabelEncoder)
Stored variables and their in-db values:
leLocation               -> LabelEncoder()
leTournament             -> LabelEncoder()


Trasformo infine la colonna ***Date*** suddividendola nelle colonne ***Day***, ***Month*** e ***Year***. 

In [11]:
dataframe['Day'] = dataframe['Date'].dt.day 
dataframe['Month'] = dataframe['Date'].dt.month 
dataframe['Year'] = dataframe['Date'].dt.year

dataframe.drop('Date', axis = 1, inplace= True)

Trasformo le feature ***Winner*** e ***Loser*** in numeriche intere:
* Ordino i giocatori in base alla differenza tra il numero delle loro vittorie e quello delle loro sconfitte;
* Assegno un numero crescente ai giocatori in base a quest'ordine.

In [12]:
players = dataframe['Winner'].value_counts().subtract(dataframe['Loser'].value_counts(), fill_value = 0).sort_values(ascending = True)

valueFromPlayer = {w:i for i,w in enumerate(players.index)}
playerFromValue = {i:w for i,w in enumerate(players.index)}

dataframe['Winner'] = dataframe['Winner'].apply(lambda x : valueFromPlayer[x])
dataframe['Loser'] = dataframe['Loser'].apply(lambda x : valueFromPlayer[x])

In [17]:
%store valueFromPlayer
%store playerFromValue

%store

Stored 'valueFromPlayer' (dict)
Stored 'playerFromValue' (dict)
Stored variables and their in-db values:
leLocation                  -> LabelEncoder()
leTournament                -> LabelEncoder()
playerFromValue             -> {0: 'Gabashvili T.', 1: 'Rochus C.', 2: 'Gimeno-Tr
valueFromPlayer             -> {'Gabashvili T.': 0, 'Rochus C.': 1, 'Gimeno-Trave


In [14]:
dataframe.head()

,csvID,ATP,Location,Tournament,Series,Round,RoundRobin,Best of,Winner,Loser,...,Hard,Awarded,Completed,Disqualified,Retired,Sched,Walkover,Day,Month,Year
0,0,1.0,2,0,1,1,0,3.0,1120,110,...,1,0,1,0,0,0,0,1,1,2001
1,0,1.0,2,0,1,2,0,3.0,1109,460,...,1,0,1,0,0,0,0,1,1,2001
2,0,1.0,2,0,1,2,0,3.0,1120,13,...,1,0,1,0,0,0,0,1,1,2001
3,0,1.0,2,0,1,2,0,3.0,959,174,...,1,0,1,0,0,0,0,1,1,2001
4,0,1.0,2,0,1,5,0,3.0,1109,1120,...,1,0,1,0,0,0,0,1,1,2001


In [19]:
%store dataframe

%store

Stored 'dataframe' (DataFrame)
Stored variables and their in-db values:
dataframe                   ->        csvID   ATP  Location  Tournament  Series  
leLocation                  -> LabelEncoder()
leTournament                -> LabelEncoder()
playerFromValue             -> {0: 'Gabashvili T.', 1: 'Rochus C.', 2: 'Gimeno-Tr
valueFromPlayer             -> {'Gabashvili T.': 0, 'Rochus C.': 1, 'Gimeno-Trave
